## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-07-02-14-31 +0000,nyt,Immigration Raid Exposes Tensions From Seoul t...,https://www.nytimes.com/2025/09/06/us/immigrat...
1,2025-09-07-02-05-27 +0000,missionlocal,Day 179 at the 16th St. Plaza: Nicolas Vance’s...,https://missionlocal.org/2025/09/day-179-at-th...
2,2025-09-07-02-00-00 +0000,wsj,Tariffs? Dutch Pensions? What’s Giving Bond Ma...,https://www.wsj.com/finance/investing/tariffs-...
3,2025-09-07-02-00-00 +0000,wsj,The Rivalry That Won’t Quit: Steph and LeBron ...,https://www.wsj.com/business/retail/the-rivalr...
4,2025-09-07-01-42-28 +0000,startribune,"Gophers score early, often in 66-0 rout of ove...",https://www.startribune.com/gophers-score-earl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
84,trump,20
112,world,8
0,immigration,8
1,raid,8
125,president,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
12,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...,52
157,2025-09-06-09-00-00 +0000,wsj,Rep. Marjorie Taylor Greene is walking a tight...,https://www.wsj.com/politics/policy/marjorie-t...,46
152,2025-09-06-09-01-55 +0000,nyt,"George Mason’s President, Gregory Washington, ...",https://www.nytimes.com/2025/09/06/us/george-m...,41
91,2025-09-06-15-27-41 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...,38
114,2025-09-06-13-00-00 +0000,wsj,Uncertainty over the fate of President Trump’s...,https://www.wsj.com/articles/u-s-importers-eye...,38


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
12,52,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...
91,38,2025-09-06-15-27-41 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
30,29,2025-09-06-22-20-46 +0000,nyt,Authorities Point to Cable Disconnecting in Fi...,https://www.nytimes.com/2025/09/06/world/europ...
85,29,2025-09-06-16-25-59 +0000,latimes,LAPD ends protection of former Vice President ...,https://www.latimes.com/california/story/2025-...
42,26,2025-09-06-20-59-15 +0000,nypost,"Israel issues evacuation orders, tells Palesti...",https://nypost.com/2025/09/06/world-news/israe...
27,26,2025-09-06-22-40-13 +0000,nyt,A World War II Dispute Between Ukraine and Pol...,https://www.nytimes.com/2025/09/06/world/europ...
22,24,2025-09-06-23-27-00 +0000,wsj,Air Canada Flight Attendants Reject Tentative ...,https://www.wsj.com/business/airlines/air-cana...
69,23,2025-09-06-17-59-05 +0000,bbc,Chris Mason: Reform conference shows party's g...,https://www.bbc.com/news/articles/c62zdpke8kko...
58,23,2025-09-06-19-08-56 +0000,nypost,At least 6 Ukrainians civilians killed overnig...,https://nypost.com/2025/09/06/world-news/at-le...
1,22,2025-09-07-02-05-27 +0000,missionlocal,Day 179 at the 16th St. Plaza: Nicolas Vance’s...,https://missionlocal.org/2025/09/day-179-at-th...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
